In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from numpy import log,dot,e,shape
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

In [ ]:
data=pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
data.sample(10)

Check the type of each column:

In [ ]:
print(data.dtypes)

getting rid of 'id' column:

In [ ]:
data.drop(labels='id', axis=1, inplace=True)

In [ ]:
data

In [ ]:
#Number of unique tokens n f_27
print(data['f_27'].nunique())

No null value

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder as le
data['f_27']=le().fit_transform(data['f_27'])
data.head()

In [ ]:
x=data.drop(labels=['target'], axis=1)
y=data['target']

In [ ]:
feat_eng(x)
x

In [ ]:
from sklearn.preprocessing import MinMaxScaler as mm
ss=mm().fit_transform(x)
ss

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ss, y, test_size=0.2, random_state=30)

Feature Eng on f_27

In [ ]:
 data['f_27'] = data['f_27'].str.upper()
# To ensure all the string characters are capitalized.

data['length'] = data['f_27'].str.len()
print(data['length'].min(), data['length'].max())

In [ ]:
def add_letters_count(data):

  letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

  for char in letters:
    data[char] = data['f_27'].str.count(char)

  return data

data = add_letters_count(data)
data.head()

In [ ]:
plt.figure(figsize=(10,10))
data.loc[:,'A':'Z'].sum().plot.bar()
plt.show()

In [ ]:
def remove_zero(data):    
  cross_check = 'UVWXYZ'
  for char in cross_check:
    if data[char].sum() == 0:
      data = data.drop([char], axis=1)
  
  return data

data = remove_zero(data)
data.head()

In [ ]:
import seaborn as sb
plt.subplots(figsize=(20,20))

for i, col in enumerate(data.loc[:,'A':'T']):
  plt.subplot(5,4, i+1)
  sb.countplot(x=data[col]>0, hue=data['target'])
plt.show()

In [ ]:
def add_pos(data):
  for i in range(10):
    data['pos' + str(i)] = (data['f_27'].str[i]).apply(lambda x: ord(x)) - 75

  return data

data = add_pos(data)
data.head()

In [ ]:
plt.figure(figsize=(15,15))
sb.heatmap(data.corr() > 0.5, cbar=False, annot=True)
plt.show()

In [ ]:
features = data.drop(['id', 'target', 'f_27', 'length'], axis=1)
target = data['target']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size = 0.01, random_state=10)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
import tensorflow
import keras
from keras import layers

In [ ]:
model = keras.Sequential([
  layers.Input(shape=(60,)),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics = ['AUC']
)

model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    verbose = 1,
                    batch_size = 128,
                    epochs = 50,
                    validation_data = (X_val, y_val))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()
history_df.loc[:,['auc','val_auc']].plot()
plt.show()

okasdb

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
df['f_27'] = df['f_27'].str.upper()
feat_eng(df)
df = add_letters_count(df)
df = remove_zero(df)
df = add_pos(df)

print(df.shape)
df.head()

In [ ]:
features = df.drop(['id', 'target', 'f_27'], axis=1)
target = df['target']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size = 0.01, random_state=10)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
model = keras.Sequential([
  layers.Input(shape=(63,)),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(),

  layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics = ['AUC']
)

model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    verbose = 1,
                    batch_size = 128,
                    epochs = 5,
                    validation_data = (X_val, y_val))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()
history_df.loc[:,['auc','val_auc']].plot()
plt.show()

test

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
df_test.head()

In [ ]:
df_test['f_27'] = df_test['f_27'].str.upper()
feat_eng(df_test)
df_test = add_letters_count(df_test)
df_test = remove_zero(df_test)
df_test = add_pos(df_test)

print(df_test.shape)
df_test.head()

In [ ]:
test = df_test.drop(['f_27', 'id'], axis=1)
test = scaler.transform(test)
predictions = model.predict(test)

In [ ]:
ss = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')
ss['target'] = np.where(predictions>0.5, 1, 0)
ss['target'].value_counts().plot.bar()
plt.show()

In [ ]:
ss['target'] = predictions

ss.to_csv('Submission.csv', index=False)
ss.head()

Log Reg

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='elasticnet',solver='saga',verbose=True,random_state=0,l1_ratio=0).fit(X_train, y_train)
clf.predict_proba(X_train)

In [ ]:
accuracy = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
print("Accuracy: %.5f" % (accuracy))
accuracy = roc_auc_score(y_test, clf.predict(X_test))
print("Accuracy: %.5f" % (accuracy))

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

In [ ]:
accuracy = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
print("Accuracy: %.5f" % (accuracy))
accuracy = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: %.5f" % (accuracy))

XGBoost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
accuracy = roc_auc_score(y_train, model.predict(X_train))
print("Train Accuracy roc: %.5f" % (accuracy))
accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train Accuracy acc: %.5f" % (accuracy))

In [ ]:
accuracy = roc_auc_score(y_val, model.predict(X_val))
print("Accuracy roc: %.5f" % (accuracy))
accuracy = accuracy_score(y_val, model.predict(X_val))
print("Accuracy acc: %.5f" % (accuracy))

**Applying pca**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
principalComponents = pca.fit_transform(x)

In [ ]:
principalDf = pd.DataFrame(data = principalComponents)
principalDf.sample(20)

In [ ]:
from sklearn.preprocessing import MinMaxScaler as mm
ss=mm().fit_transform(principalDf)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ss, y, test_size=0.2, random_state=30)
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Correlation matrix

In [ ]:
corr_mat=data.corr()
import seaborn as sns
ax = sns.heatmap(corr_mat, linewidth=0.5)
plt.show()

In [ ]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch


In [ ]:
# define the model
clf1_nopreproc = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

# fit the model 
clf1_nopreproc.fit(
    X_train,y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['auc','accuracy'],
    max_epochs=5 , patience=50,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)

In [ ]:
# plot losses
plt.plot(clf1_nopreproc.history['loss'])

# plot accuracy
plt.plot(clf1_nopreproc.history['train_accuracy'])
plt.plot(clf1_nopreproc.history['valid_accuracy'])

# find and plot feature importance
y_pred = clf1_nopreproc.predict(X_test)
clf1_nopreproc.feature_importances_
feat_importances = pd.Series(clf1_nopreproc.feature_importances_, index=data.columns[1:])
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
clf1_nopreproc.feature_importances_
feat_importances = pd.Series(clf1_nopreproc.feature_importances_, index=data.columns[1:])
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
# determine best accuracy for test set
preds = clf1_nopreproc.predict(X_test)
test_acc = roc_auc_score(preds, y_test)

# determine best accuracy for training set
preds_valid = clf1_nopreproc.predict(X_train)
valid_acc = roc_auc_score(preds_valid, y_train)

print(f"BEST ACCURACY SCORE ON TRAINING SET : {valid_acc}")
print(f"BEST ACCURACY SCORE ON TEST SET : {test_acc}")

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
test_acc_roc = roc_auc_score(y_pred, y_test)
test_acc_sco = accuracy_score(y_pred, y_test)
print(f"BEST ROC SCORE ON TEST SET : {test_acc_roc}")
print(f"BEST ACC SCORE ON TEST SET : {test_acc_sco}")

auiysbx

In [ ]:
def feat_eng(df):
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int)- (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int)- (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int)- (i_00_01_26 < -5.0).astype(int)
    

In [ ]:
feat_eng(data)
data

In [ ]:
def fnc(parent_df):
    df=parent_df.drop(labels=['id','target'], axis=1)
    df['f_27']=le().fit_transform(df['f_27'])
    feat_eng(df)
    se=mm().fit_transform(df)
    return se

In [ ]:
arr=fnc(data)
y=data['target']
X_train, X_test, y_train, y_test = train_test_split(arr, y, test_size=0.2, random_state=30)

Dealing with test data

In [ ]:
def make_sub(data_str,model):
    parent_df=pd.read_csv(data_str)
    df=parent_df.drop(labels='id', axis=1)
    df['f_27']=le().fit_transform(df['f_27'])
    se=mm().fit_transform(df)
    pred=model.predict_proba(se)[:,1]
    df2=pd.DataFrame(parent_df['id'])
    df2['target']=pred
    df2.to_csv('output.csv',index=False)

In [ ]:
make_sub('../input/tabular-playground-series-may-2022/test.csv',clf)